In [2]:
import pandas as pd
import folium 
import csv

In [3]:
bike_share_data = pd.read_csv("/Users/mac/Downloads/201902-fordgobike-tripdata.csv")
bike_share_data.head(5)

,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,member_birth_year,member_gender,bike_share_for_all_trip
0,52185,2019-02-28 17:32:10.1450,2019-03-01 08:01:55.9750,21.0,Montgomery St BART Station (Market St at 2nd St),37.789625,-122.400811,13.0,Commercial St at Montgomery St,37.794231,-122.402923,4902,Customer,1984.0,Male,No
1,42521,2019-02-28 18:53:21.7890,2019-03-01 06:42:03.0560,23.0,The Embarcadero at Steuart St,37.791464,-122.391034,81.0,Berry St at 4th St,37.775880,-122.393170,2535,Customer,NaN,NaN,No
2,61854,2019-02-28 12:13:13.2180,2019-03-01 05:24:08.1460,86.0,Market St at Dolores St,37.769305,-122.426826,3.0,Powell St BART Station (Market St at 4th St),37.786375,-122.404904,5905,Customer,1972.0,Male,No
3,36490,2019-02-28 17:54:26.0100,2019-03-01 04:02:36.8420,375.0,Grove St at Masonic Ave,37.774836,-122.446546,70.0,Central Ave at Fell St,37.773311,-122.444293,6638,Subscriber,1989.0,Other,No
4,1585,2019-02-28 23:54:18.5490,2019-03-01 00:20:44.0740,7.0,Frank H Ogawa Plaza,37.804562,-122.271738,222.0,10th Ave at E 15th St,37.792714,-122.248780,4898,Subscriber,1974.0,Male,Yes


In [ ]:
# Aggregate the data based on station name, latitude, and longitude.

In [4]:
station_data = bike_share_data.groupby(['start_station_name', 'start_station_latitude', 'start_station_longitude']).size().reset_index(name='num_rides')
station_data.head(5)

,start_station_name,start_station_latitude,start_station_longitude,num_rides
0,10th Ave at E 15th St,37.792714,-122.248780,57
1,10th St at Fallon St,37.797673,-122.262997,353
2,10th St at University Ave,37.869060,-122.293400,190
3,11th St at Bryant St,37.770030,-122.411726,828
4,11th St at Natoma St,37.773507,-122.416040,868


In [ ]:
#Calculate metrics such as the most popular stations, least popular stations, average rides per station, etc.

In [5]:
most_popular_stations = station_data.nlargest(5, 'num_rides')
least_popular_stations = station_data.nsmallest(5, 'num_rides')
average_rides_per_station = station_data['num_rides'].mean()
print( "average_rides_per_station = ",average_rides_per_station)
print( "least_popular_stations = \n" ,least_popular_stations)
print("average_rides_per_station = ",average_rides_per_station)

average_rides_per_station =  556.8844984802431
least_popular_stations = 
                  start_station_name  start_station_latitude  \
10                    16th St Depot               37.766349   
24   21st Ave at International Blvd               37.784855   
232            Palm St at Willow St               37.317298   
233     Parker Ave at McAllister St               37.776101   
326            Willow St at Vine St               37.318450   

     start_station_longitude  num_rides  
10               -122.396292          2  
24               -122.239305          4  
232              -121.884995          4  
233              -122.453093          7  
326              -121.883172          9  
average_rides_per_station =  556.8844984802431


In [ ]:
#Create a folium map centered on the city where the bike share system is located.

In [6]:

city_location = [bike_share_data['start_station_latitude'].mean(), bike_share_data['start_station_longitude'].mean()]
bike_map = folium.Map(location=city_location, zoom_start=13, size = 12)
bike_map


In [ ]:
# Plot the bike share station data on the map.

In [7]:
from folium.plugins import MarkerCluster
marker_cluster = MarkerCluster().add_to(bike_map)

for index, row in station_data.iterrows():
    station_name = row['start_station_name']
    num_rides = row['num_rides']
    latitude = row['start_station_latitude']
    longitude = row['start_station_longitude']

    # Customize marker appearance based on the number of rides
    color = 'green' if num_rides > average_rides_per_station else 'blue'
    icon = folium.Icon(color=color)

    folium.Marker(location=[latitude, longitude], popup=f"{station_name}: {num_rides} rides", icon=icon).add_to(marker_cluster)
bike_map

In [8]:
# Save the folium map as an HTML file
bike_map.save("bike_share_map.html")